In [1]:
# import subprocess
# import sys

# def install(package):
#     subprocess.check_call([sys.executable, "-m", "pip", "install", package])

In [2]:
# #packages = ['pyreadr','scikit-survival']
# install('scikit-survival')

CalledProcessError: Command '['C:\\ProgramData\\Anaconda3\\python.exe', '-m', 'pip', 'install', 'scikit-survival']' returned non-zero exit status 1.

In [1]:
# from os import listdir, getcwd
# from os.path import join

# import pyreadr

import pandas as pd
import numpy as np
#import csv

import matplotlib.pyplot as plt

#import eli5
#from eli5.sklearn import PermutationImportance

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import sksurv.metrics
from sksurv.metrics import (
    concordance_index_censored,
    concordance_index_ipcw,
    cumulative_dynamic_auc,
    integrated_brier_score,
    brier_score
)

from sksurv.functions import StepFunction
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest, GradientBoostingSurvivalAnalysis

#import LTRCmetrics as lt

# from pysurvival.models.parametric import GompertzModel
# from pysurvival.utils.metrics import concordance_index as ps_cindex
# from pysurvival.utils.display import integrated_brier_score as ps_ibs

ModuleNotFoundError: No module named 'sksurv'

# Load data

In [ ]:
# read in data
# path = "4_Predictions/time_age_comp/"
path = ''
# y_train_age = pd.read_pickle(path + "data/y_train_age.pkl")
# y_test_age = pd.read_pickle(path + "data/y_test_age.pkl")

# y_train_time = pd.read_csv(path + "4_modelling_data/y_train_ti.csv")
# y_test_time = pd.read_csv(path + "4_modelling_data/y_test_ti.csv")
y_train_time = pd.read_csv(path + "4_modelling_data/minus_backimpd/y_train_ti.csv")
y_test_time = pd.read_csv(path + "4_modelling_data/minus_backimpd/y_test_ti.csv")

x_train_time = pd.read_csv(path + "4_modelling_data/minus_backimpd/x_train_ti.csv")
x_test_time = pd.read_csv(path + "4_modelling_data/minus_backimpd/x_test_ti.csv")

print(y_train_time.head());
print(y_test_time.head())

# print(x_train_time.head());
# print(x_test_time.head())

In [ ]:
y_train_age = y_train_time.loc[:,['event','time']]
y_train_age['time'] = y_train_age['time'] + x_train_time['age']
y_train_age['event'] = y_train_age['event'].astype(bool)

In [ ]:
x_train_time['age']

In [ ]:
y_train_age

In [ ]:
np.max(y_train_age['time'])

In [ ]:
y_train_age = y_train_age.to_records(index = False);
y_train_age

# Load estimated survival curves

In [ ]:
# time points
# times = np.arange(1,27).astype(int); 
times = np.arange(1,26).astype(int); 
times

In [ ]:
# drop spouse variables?
# models: all, no_pwarm, nospouse, educl, gender, race
model = 'race'

In [ ]:
if model == 'all':
#     path = '5_predictions/curves/'
    path = '5_predictions/minus_backimpd/curves/'
    
    cphreduced_time = pd.read_csv(path + "cphreduced_out.csv") # CoxPH reduced
    cph_time = pd.read_csv(path + "cph_out.csv") # CoxPH
    cph_tv_time = pd.read_csv(path + "cph_tv_out.csv") # time-varying CPH
    #cphlasso_tv_time = pd.read_csv(path + 'cphlasso_tv_out.csv') # time-varying CPH lasso
    cphnet_time = pd.read_csv(path + "cphnet_out.csv") # CoxNet
    gbd_time = pd.read_csv(path + "gbd_out.csv") # GradBoost
    rsf_time = pd.read_csv(path + "rsf_out.csv") # RSF
    # random_time = pd.read_csv(path + "random_out.csv") # Random
    km_time = pd.read_csv(path + "km_out.csv") # Kaplan-Meier
    gomp_time = pd.read_csv(path + "gomp_out.csv") # Gompertz
    pchazard_time = pd.read_csv(path + "pchazard_time.csv") # pchazard DL
    deepsurv_time = pd.read_csv(path + "deepsurv_time.csv") # Deepsurv
    deephit_time = pd.read_csv(path + "deephit_time.csv") # Deephit
    rrf_time = pd.read_csv(path + "rrf_out.csv") # RRF
    # cif_time = pd.read_csv(path + "cif_out.csv") # CIF
    
    mods = [
        cphreduced_time,
        cph_time,
        cph_tv_time,
#         cphlasso_tv_time,
        cphnet_time,
        gbd_time,
        rsf_time,
#         random_time,
        km_time,
        gomp_time,
        pchazard_time,
        deepsurv_time,
        deephit_time,
#         cif_time,        
        rrf_time
    ]
    names = [
        'CoxReduced',
        'Cox',
        'Cox-TV',
        #'CoxLasso-TV',
        'CoxNet',
        'GradBoost',
        'RSF',
        #'Random',
        'KM',
        'Gompertz',
        'DeepPCH',
        'DeepSurv',
        'DeepHit',
        #'CIF-TV',
        'RRF-TV'
    ]    
        
    for mod in mods:
        print(mod.shape)
        
    keepcols = np.intersect1d(cph_time.drop(['time'],axis=1).columns,cph_tv_time.drop(['time'],axis=1).columns)
    print(keepcols.shape) 
    
elif model == 'no_pwarm':
    path = '5_predictions/no_pwarm/curves/'
    
    cphreduced_time = pd.read_csv(path + "cphreduced_out.csv") # CoxPH reduced
    cph_time = pd.read_csv(path + "cph_out.csv") # CoxPH
    cph_tv_time = pd.read_csv(path + "cph_tv_out.csv") # time-varying CPH
    #cphlasso_tv_time = pd.read_csv(path + 'cphlasso_tv_out.csv') # time-varying CPH lasso
    cphnet_time = pd.read_csv(path + "cphnet_out.csv") # CoxNet
    gbd_time = pd.read_csv(path + "gbd_out.csv") # GradBoost
    rsf_time = pd.read_csv(path + "rsf_out.csv") # RSF
    # random_time = pd.read_csv(path + "random_out.csv") # Random
    km_time = pd.read_csv(path + "km_out.csv") # Kaplan-Meier
    gomp_time = pd.read_csv(path + "gomp_out.csv") # Gompertz
    pchazard_time = pd.read_csv(path + "pchazard_nopwarm.csv") # pchazard DL
    deepsurv_time = pd.read_csv(path + "deepsurv_nopwarm.csv") # Deepsurv
    deephit_time = pd.read_csv(path + "deephit_nopwarm.csv") # Deephit
    rrf_time = pd.read_csv(path + "rrf_out.csv") # RRF
    # cif_time = pd.read_csv(path + "cif_out.csv") # CIF
    
    mods = [
        cphreduced_time,
        cph_time,
        cph_tv_time,
#         cphlasso_tv_time,
        cphnet_time,
        gbd_time,
        rsf_time,
#         random_time,
        km_time,
        gomp_time,
        pchazard_time,
        deepsurv_time,
        deephit_time,
#         cif_time,        
        rrf_time
    ]
    names = [
        'CoxReduced',
        'Cox',
        'Cox-TV',
        #'CoxLasso-TV',
        'CoxNet',
        'GradBoost',
        'RSF',
        #'Random',
        'KM',
        'Gompertz',
        'DeepPCH',
        'DeepSurv',
        'DeepHit',
        #'CIF-TV',
        'RRF-TV'
    ]    
        
    for mod in mods:
        print(mod.shape)
        
    keepcols = np.intersect1d(cph_time.drop(['time'],axis=1).columns,cph_tv_time.drop(['time'],axis=1).columns)
    print(keepcols.shape)       
    
elif model == 'nospouse':    
    path = '5_predictions/nospouse/curves/'
    
    cphreduced_time = pd.read_csv(path + "cphreduced_out.csv") # CoxPH reduced
    cph_time = pd.read_csv(path + "cph_out.csv") # CoxPH
    cph_tv_time = pd.read_csv(path + "cph_tv_out.csv") # time-varying CPH
    #cphlasso_tv_time = pd.read_csv(path + 'cphlasso_tv_out.csv') # time-varying CPH lasso
    cphnet_time = pd.read_csv(path + "cphnet_out.csv") # CoxNet
    gbd_time = pd.read_csv(path + "gbd_out.csv") # GradBoost
    rsf_time = pd.read_csv(path + "rsf_out.csv") # RSF
    # random_time = pd.read_csv(path + "random_out.csv") # Random
    km_time = pd.read_csv(path + "km_out.csv") # Kaplan-Meier
    gomp_time = pd.read_csv(path + "gomp_out.csv") # Gompertz
    pchazard_time = pd.read_csv(path + "pchazard_rlevel.csv") # pchazard DL
    deepsurv_time = pd.read_csv(path + "deepsurv_rlevel.csv") # Deepsurv
    deephit_time = pd.read_csv(path + "deephit_rlevel.csv") # Deephit
    rrf_time = pd.read_csv(path + "rrf_out.csv") # RRF
    # cif_time = pd.read_csv(path + "cif_out.csv") # CIF
    
    mods = [
        cphreduced_time,
        cph_time,
        cph_tv_time,
#         cphlasso_tv_time,
        cphnet_time,
        gbd_time,
        rsf_time,
#         random_time,
        km_time,
        gomp_time,
        pchazard_time,
        deepsurv_time,
        deephit_time,
#         cif_time,  
        rrf_time
    ]
    names = [
        'CoxReduced',        
        'Cox',
        'Cox-TV',
        #'CoxLasso-TV',
        'CoxNet',
        'GradBoost',
        'RSF',
        #'Random',
        'KM',
        'Gompertz',
        'DeepPCH',
        'DeepSurv',
        'DeepHit',
        #'CIF-TV',
        'RRF-TV'
    ] 
    
    for mod in mods:
        print(mod.shape)  
        
    keepcols = np.intersect1d(cph_time.drop(['time'],axis=1).columns,cph_tv_time.drop(['time'],axis=1).columns)
    print(keepcols.shape)   
    
else:
#     path = '5_predictions/oversampled/curves/' + model + "/"      
    path = '5_predictions/oversampled_minus_backimpd/curves/' + model + "/"    

    cphreduced_time = pd.read_csv(path + "cphreduced_out.csv") # CoxPH reduced    
    cph_time = pd.read_csv(path + "cph_out.csv") # CoxPH
    cphnet_time = pd.read_csv(path + "cphnet_out.csv") # CoxNet
    gbd_time = pd.read_csv(path + "gbd_out.csv") # GradBoost
    rsf_time = pd.read_csv(path + "rsf_out.csv") # RSF
    # random_time = pd.read_csv(path + "random_out.csv") # Random
    km_time = pd.read_csv(path + "km_out.csv") # Kaplan-Meier
#     gomp_time = pd.read_csv(path + "gomp_out_" + model + ".csv") # Gompertz
    gomp_time = pd.read_csv(path + "gomp_out.csv") # Gompertz
    pchazard_time = pd.read_csv(path + "pchazard_" + model + "_time.csv") # pchazard DL
    deepsurv_time = pd.read_csv(path + "deepsurv_" + model + "_time.csv") # Deepsurv
    deephit_time = pd.read_csv(path + "deephit_" + model + "_time.csv") # Deephit
    
    mods = [
        cphreduced_time,
        cph_time,
        cphnet_time,
        gbd_time,
        rsf_time,
#         random_time,
        km_time,
        gomp_time,
        pchazard_time,
        deepsurv_time,
        deephit_time
    ]
    names = [
        'CoxReduced',
        'Cox',
        'CoxNet',
        'GradBoost',
        'RSF',
        #'Random',
        'KM',
        'Gompertz',
        'DeepPCH',
        'DeepSurv',
        'DeepHit'
    ]    
    
    for mod in mods:
        print(mod.shape)   
        
    keepcols = np.intersect1d(cph_time.drop(['time'],axis=1).columns,gomp_time.drop(['time'],axis=1).columns)
    print(keepcols.shape) 

In [ ]:
probs = [np.array(mod.loc[times,keepcols].transpose()) for mod in mods]    
probs_full = [np.array(mod.loc[:,keepcols].transpose()) for mod in mods]   

# Evaluate Performance

## Form list of predictions

In [ ]:
# # check all time evaluation points are the same
# times_cph = np.array(cph_time['time']); print(times_cph)
# times_cphnet = np.array(cphnet_time['time']); print(times_cphnet)
# times_rsf = np.array(rsf_time['time']); print(times_rsf)
# times_gbd = np.array(gbd_time['time']); print(times_gbd)
# times_random = np.array(random_time['time']); print(times_random)
# times_km = np.array(km_time['time']); print(times_km)
# times_gomp = np.array(gomp_time['time']); print(times_gomp)
# times_pchazard = np.array(pchazard_time['time']); print(times_pchazard)
# times_deepsurv = np.array(deepsurv_time['time']); print(times_deepsurv)
# times_deephit = np.array(deephit_time['time']); print(times_deephit)
# times_rrf = np.array(rrf_time['time']); print(times_rrf)
# # times_cif = np.array(cif_time['time']); print(times_cif)
# times_cph_tv = np.array(cph_tv_time['time']); print(times_cph_tv)
# times_cphlasso_tv = np.array(cphlasso_tv_time['time']); print(times_cphlasso_tv)

In [ ]:
# times = times_cph[0:28].astype(int); times # can't start at 0

In [ ]:
# times = times_cph[1:27].astype(int); times

In [ ]:
# times = np.arange(1,27).astype(int); times
# times = np.arange(1,26).astype(int); times

In [ ]:
for j in range(len(probs)):
    print(probs[j].shape)
    print(probs_full[j].shape)

In [ ]:
# remove subjects fromt training set with 0 time in study
# train_inds = (y_train_time['exit_time_outcome'] >= 0); train_inds

In [ ]:
# np.sum(y_train_time['exit_time_outcome'] == 0)

In [ ]:
# remove subjects from test set with time in study greater than max time in train set
# test_inds = (y_test_time['exit_time_outcome'] <= np.max(y_train_time['exit_time_outcome'])) & (y_test_time['exit_time_outcome'] >= 0)
# np.sum(1-test_inds)

In [ ]:
y_test_time.index = x_test_time['hhidpn'].astype(str)

In [ ]:
y_test_time.loc[keepcols.tolist()]

In [ ]:
y_train_time['event'] = y_train_time['event'].astype(bool)
y_test_time['event'] = y_test_time['event'].astype(bool)

In [ ]:
y_train = y_train_time.loc[:,['event','time']].to_records(index=False)
y_test = y_test_time.loc[keepcols.tolist(),['event','time']].to_records(index=False)

In [ ]:
y_train

In [ ]:
print(y_train.dtype)
print(y_test.dtype)

print(y_train.shape)
print(y_test.shape)

print(y_train_time.shape)
print(y_test_time.shape)

In [ ]:
np.sum(y_test_time['time'] >= 27) # very small sample size beyong 27 -- evaluate metrics only up to 26

#### Include sample weights and standard error calculations in metrics?

In [ ]:
# def brier_score_weights(y_train, y_test, probs, times, weights, z):

# def integrated_brier_score_weights(y_train, y_test, probs, times, weights, z):

# def cumulative_dynamic_auc_weights(y_train, y_test, risks, times, weights, z):

#### Calculate MAE of predicted years lived in study

In [ ]:
def rmse(obs,exp,inds):
    return(np.sqrt(np.mean(inds*((obs-exp)**2))))

def mae(obs,exp,inds):
    return(np.mean(np.abs(inds*(obs-exp))))

def bias(obs,exp,inds):
    return(np.mean(inds*(obs-exp)))

In [ ]:
# dicts = {}
# for ind, model in enumerate(names):
#         dicts[model] = np.trapz(probs_full[ind],axis=1) 
        
# dicts

In [ ]:
expected_years = pd.Series(
    [np.trapz(probs_full[ind],axis=1) for ind in range(len(names))],
    index = names,
    name = "Expected years lived"
)

## Calculate metrics

In [ ]:
# Compare models
score_int_brier = pd.Series(
    [integrated_brier_score(y_train, y_test, prob, times) for prob in probs],
    index = names,
    name = "IBS"
)

score_mean_auc = pd.Series(
    [cumulative_dynamic_auc(y_train, y_test, -prob, times)[1] for prob in probs],
    index = names,
    name = "Mean AUC"
)

score_mae = pd.Series(
    [
        mae(y_test['time'],expected_years[model],y_test['event']) for model in names
    ],
    index = names,
    name = "MAE"
)

score_bias = pd.Series(
    [
        bias(y_test['time'],expected_years[model],y_test['event']) for model in names
    ],
    index = names,
    name = "Bias"
)

# score_rmse = pd.Series(
#     [
#         rmse(y_test['time'],expected_years[model],y_test['event']) for model in names
#     ],
#     index = names,
#     name = "RMSE"
# )

# dynamic metrics
score_dynamic_brier = pd.Series(
    [brier_score(y_train, y_test, prob, times)[1] for prob in probs],
    index = names,
    name = "Brier score"
)

score_dynamic_auc = pd.Series(
    [cumulative_dynamic_auc(y_train, y_test, -prob, times)[0] for prob in probs],
    index = names,
    name = "AUC"
)

scores_rc = pd.concat([score_int_brier,score_mean_auc,score_mae,score_bias], axis = 1).round(3); scores_rc

In [ ]:
# Plot Dynamic AUC
fs = 12  # fontsize
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,6))
ax[0].set_title('time-dependent AUC', fontsize=fs)
ax[1].set_title('time-dependent Brier score', fontsize=fs)

for j in range(len(probs)):
    ax[0].plot(times, score_dynamic_auc[j], "o-", label=str(names[j]) + " (mean AUC = {:.3f})".format(score_mean_auc[j]))
    ax[1].plot(times, score_dynamic_brier[j], "o-", label=str(names[j]) + " (IBS = {:.3f})".format(score_int_brier[j]))

ax[0].set_xlabel("Time to death (yrs)")
ax[1].set_xlabel("Time to death (yrs)")

ax[0].set_ylabel("AUC")
ax[1].set_ylabel("Brier score")

ax[0].legend(loc="lower right")
ax[1].legend(loc="lower right")
ax[0].grid(True); ax[1].grid(True)

In [ ]:
path = '6_Evaluation/minus_backimpd/'

# save metrics
dicts = {'time':times}
for _, mod in enumerate(score_dynamic_brier.index):
        dicts[mod] = score_dynamic_brier[mod]
score_dynamic_brier_df = pd.DataFrame(dicts)

score_dynamic_brier_df.to_csv(path + model + '/brier.csv')

dicts = {'time':times}
for _, mod in enumerate(score_dynamic_brier.index):
        dicts[mod] = score_dynamic_auc[mod]
score_dynamic_auc_df = pd.DataFrame(dicts)

score_dynamic_auc_df.to_csv(path + model + '/auc.csv')

score_int_brier.to_csv(path + model + '/int_brier.csv')
score_mean_auc.to_csv(path + model + '/mean_auc.csv')
score_mae.to_csv(path + model + '/mae.csv')
score_bias.to_csv(path + model + '/bias.csv')

In [ ]:
score_dynamic_auc['Cox']

In [ ]:
score_dynamic_auc['GradBoost']

# Break down metrics by groups

In [ ]:
# y_train_time = y_train
# y_test_time = y_test

In [ ]:
x_train_time.index = x_train_time['hhidpn'].astype(str)
x_test_time.index = x_test_time['hhidpn'].astype(str)

In [ ]:
x_test = x_test_time.loc[keepcols.tolist(),:]
x_train = x_train_time

In [ ]:
# surv_prob_all = pyreadr.read_r("surv_prob_all.rds")[None]

In [ ]:
# surv_prob_all

In [ ]:
x_test.columns

In [ ]:
probs[0].shape

### Gender

In [ ]:
print(np.mean(y_test['time'][x_test['r_gender_cat']==0])) # men
print(np.mean(y_test['time'][x_test['r_gender_cat']==1])) # women

In [ ]:
gender = 0

train_inds = (x_train['r_gender_cat'] == gender) 
test_inds = (x_test['r_gender_cat'] == gender) 
test_inds_mae = test_inds

test_inds *=  y_test['time'] <= np.max(y_train['time'][train_inds])

if gender == 0:
    group = 'gender_men'
else:
    group = 'gender_women'

### Race

In [ ]:
print(np.mean(y_test['time'][(x_test['r_race_cat']==1) & y_test['event']])) # white
print(np.mean(y_test['time'][(x_test['r_race_cat']==2) & y_test['event']])) # black
print(np.mean(y_test['time'][(x_test['r_race_cat']==3) & y_test['event']])) # hispanic

In [ ]:
print(np.mean(y_test['time'][(x_test['r_race_cat']==1)])) # white
print(np.mean(y_test['time'][(x_test['r_race_cat']==2)])) # black
print(np.mean(y_test['time'][(x_test['r_race_cat']==3)])) # hispanic

In [ ]:
print(np.std(y_test['time'][(x_test['r_race_cat']==1) & y_test['event']])) # white
print(np.std(y_test['time'][(x_test['r_race_cat']==2) & y_test['event']])) # black
print(np.std(y_test['time'][(x_test['r_race_cat']==3) & y_test['event']])) # hispanic

In [ ]:
race1 = 1
race2 = 1

train_inds = (x_train['r_race_cat'] == race1) | (x_train['r_race_cat'] == race2)
test_inds = (x_test['r_race_cat'] == race1) | (x_test['r_race_cat'] == race2)
# train_inds = (x_train['r_race_cat'] == race1) 
# test_inds = (x_test['r_race_cat'] == race1)
test_inds_mae = test_inds

test_inds *=  y_test['time'] <= np.max(y_train['time'][train_inds])

if (race1 == 1) & (race2 == 1):
    group = 'race_white'
elif (race1 == 2) & (race2 == 2):
    group = 'race_black'
elif (race1 == 3) & (race2 == 3):
    group = 'race_hispanic' 
elif (race1 == 2) & (race2 == 3):
    group = 'race_nonwhite'      

### Age

In [ ]:
# train_inds = (x_train['age'] < 150 ) & (x_train['age'] >= 80)
# test_inds = (x_test['age'] < 150 ) & (x_test['age'] >= 80)
# test_inds_mae = test_inds

# test_inds *=  y_test['time'] <= np.max(y_train['time'][train_inds])

### Education

In [ ]:
x_train['r_educl_cat'] # 1 = low, 2 = medium, 3 = high

In [ ]:
educ = 1

train_inds = (x_train['r_educl_cat'] == educ) 
test_inds = (x_test['r_educl_cat'] == educ) 
test_inds_mae = test_inds

test_inds *=  y_test['time'] <= np.max(y_train['time'][train_inds])

if educ == 1:
    group = 'educl_low'
elif educ == 2:
    group = 'educl_medium'
else:
    group = 'educl_high'

## Calculate

In [ ]:
probs[0][test_inds,:].shape

In [ ]:
#train_inds_time = times < np.max(y_train_time['exit_time_outcome'][train_inds])
#test_inds_time = train_inds_time * (times < np.max(y_test_time['exit_time_outcome'][test_inds]))
test_inds_time = times < np.max(y_test['time'][test_inds])

In [ ]:
print(np.max(times[test_inds_time])); #print(np.max(times[train_inds_time]))

In [ ]:
print(np.max(y_test['time'][test_inds])); print(np.max(y_train['time'][train_inds]))

#### MAE expected years

In [ ]:
expected_years = pd.Series(
    [np.trapz(probs_full[ind],axis=1) for ind in range(len(names))],
    index = names,
    name = "Expected years lived"
)

In [ ]:
# Compare models
score_int_brier = pd.Series(
    [integrated_brier_score(y_train[train_inds], #,train_inds_time], 
                            y_test[test_inds],#,test_inds_time], 
                            prob[test_inds][:,test_inds_time], 
                            times[test_inds_time]) for prob in probs],
    index = names,
    name = "IBS"
)

score_mean_auc = pd.Series(
    [cumulative_dynamic_auc(y_train[train_inds], #,train_inds_time], 
                            y_test[test_inds],#,test_inds_time], 
                            -prob[test_inds][:,test_inds_time], 
                            times[test_inds_time])[1] for prob in probs],
    index = names,
    name = "Mean AUC"
)

score_mae = pd.Series(
    [
        mae(y_test['time'][test_inds_mae],
            expected_years[model][test_inds_mae],
            y_test['event'][test_inds_mae]) for model in names
    ],
    index = names,
    name = "MAE"
)

score_bias = pd.Series(
    [
        bias(y_test['time'][test_inds_mae],
            expected_years[model][test_inds_mae],
            y_test['event'][test_inds_mae]) for model in names
    ],
    index = names,
    name = "Bias"
)

# dynamic metrics
score_dynamic_brier = pd.Series(
    [brier_score(y_train[train_inds], #,train_inds_time], 
                 y_test[test_inds],#,test_inds_time], 
                 prob[test_inds][:,test_inds_time], 
                 times[test_inds_time])[1] for prob in probs],
    index = names,
    name = "Brier score"
)

score_dynamic_auc = pd.Series(
    [cumulative_dynamic_auc(y_train[train_inds], #,train_inds_time], 
                            y_test[test_inds],#,test_inds_time], 
                            -prob[test_inds][:,test_inds_time], 
                            times[test_inds_time])[0] for prob in probs],
    index = names,
    name = "AUC"
)

scores_rc = pd.concat([score_int_brier,score_mean_auc,score_mae,score_bias], axis = 1).round(3); scores_rc

In [ ]:
# Plot Dynamic AUC
fs = 12  # fontsize
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,6))
ax[0].set_title('time-dependent AUC', fontsize=fs)
ax[1].set_title('time-dependent Brier score', fontsize=fs)

for j in range(len(probs)):
    ax[0].plot(times[test_inds_time], score_dynamic_auc[j], "o-", label=str(names[j]) + " (mean AUC = {:.3f})".format(score_mean_auc[j]))
    ax[1].plot(times[test_inds_time], score_dynamic_brier[j], "o-", label=str(names[j]) + " (IBS = {:.3f})".format(score_int_brier[j]))

ax[0].set_xlabel("Time to death (yrs)")
ax[1].set_xlabel("Time to death (yrs)")

ax[0].set_ylabel("AUC")
ax[1].set_ylabel("Brier score")

ax[0].legend(loc="lower right")
ax[1].legend(loc="lower right")
ax[0].grid(True); ax[1].grid(True)

In [ ]:
path = '6_Evaluation/minus_backimpd/'

# save metrics
dicts = {'time':times}
for _, mod in enumerate(score_dynamic_brier.index):
        dicts[mod] = score_dynamic_brier[mod]
score_dynamic_brier_df = pd.DataFrame(dicts)

score_dynamic_brier_df.to_csv(path + model + '/brier_' + group +'.csv')

dicts = {'time':times}
for _, mod in enumerate(score_dynamic_brier.index):
        dicts[mod] = score_dynamic_auc[mod]
score_dynamic_auc_df = pd.DataFrame(dicts)

score_dynamic_auc_df.to_csv(path + model + '/auc_' + group +'.csv')

score_int_brier.to_csv(path + model + '/int_brier_' + group +'.csv')
score_mean_auc.to_csv(path + model + '/mean_auc_' + group +'.csv')
score_mae.to_csv(path + model + '/mae_' + group +'.csv')
score_bias.to_csv(path + model + '/bias_' + group +'.csv')

In [ ]:
score_dynamic_brier[0]

# Metrics for subjective probabilities

In [ ]:
expec = pyreadr.read_r("7_Expectations/expec_brier.rds")[None]

In [ ]:
print(expec.shape)
expec.head()

In [ ]:
pd.unique(expec['hhidpn']).shape

In [ ]:
y_train

In [ ]:
models_subj = pd.unique(expec['model'])
models_subj

In [ ]:
pd.unique(expec['outcome'])

In [ ]:
np.logical_not(np.isnan(expec['surv_prob']))

In [ ]:
expec_obs = expec[np.logical_not(np.isnan(expec['surv_prob']))]
expec_obs.shape

In [ ]:
# form test outcomes
# y_subj = [expec_obs.groupby('hhidpn').agg({'event_type_outcome':'min','exit_age_outcome':'min'})
y_subj = [expec_obs.loc[expec_obs['model'] == mod,['event_type_outcome','exit_age_outcome']] for mod in models_subj]

In [ ]:
y_subj

In [ ]:
for count,mod in enumerate(models_subj):
    y_subj[count]['event_type_outcome'] = y_subj[count]['event_type_outcome'].astype(bool)
    y_subj[count] = y_subj[count].to_records(index = False)

y_subj

In [ ]:
# form probs
probs_subj = [expec_obs[expec_obs['model'] == mod]['surv_prob'] for mod in models_subj]

In [ ]:
probs_subj

In [ ]:
for count,mod in enumerate(models_subj):
    print(probs_subj[count].shape)

In [ ]:
# Compare models
score_brier = pd.Series(
    [brier_score(y_train_age,
                 y_subj[j],
                 probs_subj[j], 
                 75)[1][0] for j in range(len(models_subj))],
    index = models_subj,
    name = "Brier score"
)

score_auc = pd.Series(
    [cumulative_dynamic_auc(y_train_age,
                 y_subj[j],
                 -probs_subj[j], 
                 75)[0][0] for j in range(len(models_subj))],
    index = models_subj,
    name = "AUC"
)

scores = pd.concat([score_brier,score_auc], axis = 1).round(3); scores

# RMSE of observed vs. expected years lived in study

## Calculate expected years lived in study

In [ ]:
y_test_dead = y_test[y_test['event'] == 1]

In [ ]:
dicts = {}
for ind, model in enumerate(names):
        dicts[model] = np.sum(probs[ind],axis=1) 
        
dicts

In [ ]:
np.sum(probs[10],axis=1).shape

In [ ]:
expected_years = pd.Series(
    [np.sum(probs[ind],axis=1) for ind in range(len(names))],
    index = names,
    name = "Expected years lived"
)

In [ ]:
expected_years['Cox']

## Calculate RMSE

In [ ]:
def rmse(obs,exp,inds):
    return(np.sqrt(np.mean(inds*((obs-exp)**2))))

def mae(obs,exp,inds):
    return(np.mean(np.abs(inds*(obs-exp))))

In [ ]:
rmse_years = pd.Series(
    [
        rmse(y_test['time'],expected_years[model],y_test['event']) for model in names
    ],
    index = names,
    name = "RMSE of predicted years lived"
)

In [ ]:
rmse_years.round(2)

# Plot survival curves

In [ ]:
# plot some estimated survival curves
plt.plot(times,cph_surv_prob.transpose()[:,0:10])

# Legacy code

## Misc data exploration

In [ ]:
# read in data
df_train = pyreadr.read_r("4_modelling_data/all_train_imputed_scaled.rds")[None]
df_test = pyreadr.read_r("4_modelling_data/all_test_imputed_scaled.rds")[None]

In [ ]:
df_train['event_type_outcome'] = df_train['event_type_outcome'].astype(bool)
df_test['event_type_outcome'] = df_test['event_type_outcome'].astype(bool)

In [ ]:
print(df_train.shape)
df_train.head()

In [ ]:
df_train.columns[0:100]

In [ ]:
df_train['start_outcome']

In [ ]:
np.unique(df_test['hhidpn']).shape

In [ ]:
df_test.groupby('hhidpn').agg({'event_type_outcome': 'max', 'exit_time_outcome': 'max'})

In [ ]:
inds = np.where((df_train['exit_time_outcome'] < 1) & 
                (df_train['exit_time_outcome'] > 0) &
                (df_train['event_type_outcome'] == 0))
plt.hist(df_train.iloc[inds]['last_wave_outcome'])
plt.hist(df_train.iloc[inds]['first_wave_outcome'])

In [ ]:
df_train.columns[0:30]

In [ ]:
df_train.iloc[inds]['iwdate_outcome']

In [ ]:
inds = np.where((df_train['exit_time_outcome'] == 0))
plt.hist(df_train.iloc[inds]['last_wave_outcome'] == df_train.iloc[inds]['first_wave_outcome'])

## Create survival objects

In [ ]:
# y_train = df_train[df_train['last_wave_outcome'] == df_train['wave_outcome']].loc[:,['event_type_outcome','exit_time_outcome']].to_records(index=False)
# y_test = df_test[df_test['last_wave_outcome'] == df_test['wave_outcome']].loc[:,['event_type_outcome','exit_time_outcome']].to_records(index=False)

In [ ]:
# y_train = df_train[df_train['age'] == df_train['start_outcome']].loc[:,['event_type_outcome','exit_time_outcome']].to_records(index=False)
# y_test = df_test[df_test['age'] == df_test['start_outcome']].loc[:,['event_type_outcome','exit_time_outcome']].to_records(index=False)

In [ ]:
y_train = df_train.groupby('hhidpn').agg({'event_type_outcome': 'max', 'exit_time_outcome': 'max'}).loc[:,['event_type_outcome','exit_time_outcome']].to_records(index=False)
y_test = df_test.groupby('hhidpn').agg({'event_type_outcome': 'max', 'exit_time_outcome': 'max'}).loc[:,['event_type_outcome','exit_time_outcome']].to_records(index=False)

In [ ]:
y_train

In [ ]:
y_test.shape

In [ ]:
y_train_time = y_train
y_test_time = y_test

## Load survival objects

In [ ]:
x_train_time = pd.read_pickle(path + "data/X_train_time.pkl")
x_test_time = pd.read_pickle(path + "data/X_test_time.pkl")

x_train_age = pd.read_pickle(path + "data/X_train_age.pkl")
x_test_age = pd.read_pickle(path + "data/X_test_age.pkl")

In [ ]:
# # write data to csv
# import pickle as pkl
# import pandas as pd

# with open(path + "data/y_test_age.pkl", "rb") as f:
#     object = pkl.load(f)
    
# df = pd.DataFrame(object)
# df.to_csv(path + "data/y_test_age.csv")

In [ ]:
# add left truncation times
y_train_age_lt = pd.DataFrame(y_train_age)
y_train_age_lt['Start'] = np.array(x_train_time['age'])
y_train_age_lt['Stop'] = np.array(y_train_age_lt['exit_age_outcome'])
y_train_age_lt['Event'] = np.array(y_train_age_lt['event_type_outcome'])
y_train_age_lt = y_train_age_lt.loc[:,['Start','Stop','Event']].to_records(index=False)

y_test_age_lt = pd.DataFrame(y_test_age)
y_test_age_lt['Start'] = np.array(x_test_time['age'])
y_test_age_lt['Stop'] = np.array(y_test_age_lt['exit_age_outcome'])
y_test_age_lt['Event'] = np.array(y_test_age_lt['event_type_outcome'])
y_test_age_lt = y_test_age_lt.loc[:,['Start','Stop','Event']].to_records(index=False)

In [ ]:
y_train_age_lt

In [ ]:
# add left truncation times
y_train_time_lt = pd.DataFrame(y_train_time)
y_train_time_lt['Start'] = 0 
y_train_time_lt['Stop'] = np.array(y_train_time_lt['exit_time_outcome'])
y_train_time_lt['Event'] = np.array(y_train_time_lt['event_type_outcome'])
y_train_time_lt = y_train_time_lt.loc[:,['Start','Stop','Event']].to_records(index=False)

y_test_time_lt = pd.DataFrame(y_test_time)
y_test_time_lt['Start'] = 0
y_test_time_lt['Stop'] = np.array(y_test_time_lt['exit_time_outcome'])
y_test_time_lt['Event'] = np.array(y_test_time_lt['event_type_outcome'])
y_test_time_lt = y_test_time_lt.loc[:,['Start','Stop','Event']].to_records(index=False)

In [ ]:
y_train_time_lt

In [ ]:
# add left truncation times
y_train_time_age_lt = pd.DataFrame(y_train_age)
y_train_time_age_lt['Start'] = np.array(x_train_time['age'])
y_train_time_age_lt['Stop'] = np.array(y_train_time['exit_time_outcome'] + x_train_time['age'])
y_train_time_age_lt['Event'] = np.array(y_train_time_age_lt['event_type_outcome'])
y_train_time_age_lt = y_train_time_age_lt.loc[:,['Start','Stop','Event']].to_records(index=False)

y_test_time_age_lt = pd.DataFrame(y_test_age)
y_test_time_age_lt['Start'] = np.array(x_test_time['age'])
y_test_time_age_lt['Stop'] = np.array(y_test_time['exit_time_outcome'] + x_test_time['age'])
y_test_time_age_lt['Event'] = np.array(y_test_time_age_lt['event_type_outcome'])
y_test_time_age_lt = y_test_time_age_lt.loc[:,['Start','Stop','Event']].to_records(index=False)

In [ ]:
y_train_time_age_lt

## sanity check about age vs. time in study

In [ ]:
np.mean(np.abs(x_train_time['age'] + y_train_time['exit_time_outcome'] - y_train_age['exit_age_outcome']))

In [ ]:
plt.hist(np.abs(x_train_time['age'] + y_train_time['exit_time_outcome'] - y_train_age['exit_age_outcome']))

In [ ]:
np.mean(np.abs((df_train['iwdate_outcome'] - df_train['birth_date_outcome']) - x_train_time['age']))

In [ ]:
df_train.columns[0:30]